In [31]:
import spacy
import pandas as pd

## TRATAMIENTO DE DATOS

In [10]:
df_film = pd.read_csv("../data/film_reviews_result/film_reviews_result.csv", sep = "|")

In [11]:
df_filmafinity = pd.read_csv("../data/reviews_filmaffinity/reviews_filmaffinity.csv", sep=r"\s*\|\|\s*", engine='python')

In [12]:
df_film.head()

,film_name,gender,film_avg_rate,review_rate,review_title,review_text
0,Respira (Serie de TV),"Serie de TV, Drama, Medicina","4,6",1,Respira,"Como fan de las series españolas y de Najwa, e..."
1,Respira (Serie de TV),"Serie de TV, Drama, Medicina","4,6",2,Esta serie te llama tonto a la cara,Todo lo malo que puede tener una serie lo pose...
2,Respira (Serie de TV),"Serie de TV, Drama, Medicina","4,6",1,Respira,"La serie es un drama médico que intenta ""copia..."
3,Respira (Serie de TV),"Serie de TV, Drama, Medicina","4,6",1,"Imposible, no hay manera",Nadie te obliga a ver nada que no quieras ver ...
4,Respira (Serie de TV),"Serie de TV, Drama, Medicina","4,6",1,Serie nefasta,Está serie da vergüenza ajena. Una serie donde...


In [13]:
df_filmafinity.head()

,film_name,gender,film_avg_rate,review_rate,review_title,review_text
0,Ocho apellidos vascos,Comedia,"6,0",3.0,OCHO APELLIDOS VASCOS...Y NINGÚN NOMBRE PROPIO,La mayor virtud de esta película es su existen...
1,Ocho apellidos vascos,Comedia,"6,0",2.0,El perro verde,"No soy un experto cinéfilo, pero pocas veces m..."
2,Ocho apellidos vascos,Comedia,"6,0",2.0,Si no eres de comer mierda... no te comas esta...,Si no eres un incondicional del humor estilo T...
3,Ocho apellidos vascos,Comedia,"6,0",2.0,Aida: The movie,"No sé qué está pasando, si la gente se deja ll..."
4,Ocho apellidos vascos,Comedia,"6,0",2.0,UN HOMBRE SOLO (Julio Iglesias 1987),"""Pero cuando amanece,y me quedo solo,siento en..."


In [14]:
combined_df = pd.concat([df_film, df_filmafinity], ignore_index=True)

In [15]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18661 entries, 0 to 18660
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   film_name      18661 non-null  object 
 1   gender         18661 non-null  object 
 2   film_avg_rate  18661 non-null  object 
 3   review_rate    18659 non-null  float64
 4   review_title   18661 non-null  object 
 5   review_text    18661 non-null  object 
dtypes: float64(1), object(5)
memory usage: 874.9+ KB


In [16]:
combined_df.isna().sum()

film_name        0
gender           0
film_avg_rate    0
review_rate      2
review_title     0
review_text      0
dtype: int64

In [17]:
combined_df.dropna(inplace=True)

In [18]:
combined_df.isna().sum()

film_name        0
gender           0
film_avg_rate    0
review_rate      0
review_title     0
review_text      0
dtype: int64

## NORMALIZACIÓN DE LOS DATOS

In [36]:
nlp = spacy.load('es_core_news_sm')

c:\Users\Ryzen\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'es_core_news_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.6). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [37]:
def tokenization(texto):
  doc = nlp(texto)
  tokens = [token.text.lower() for token in doc]
  return tokens

In [38]:
def delete_stopwords(tokens):
  filter_tokens = [token for token in tokens if not token.is_punct and not token.is_stop]
  return filter_tokens

In [39]:
def lemmatization(tokens):
  lemmas = [token.lemma_ for token in tokens]
  return lemmas

In [40]:
def normalization(doc):
  tokens = tokenization(doc)
  tokens = nlp(' '.join(tokens))
  tokens = delete_stopwords(tokens)
  tokens = lemmatization(tokens)
  return tokens

In [41]:
combined_df['review_title'].apply(normalization)

0                                               [respirar]
1                             [serie, llamar, tonto, cara]
2                                               [respirar]
3                                              [imposible]
4                                         [serie, nefasto]
                               ...                        
18656                                          [magnífico]
18657    [bombo, español, cualquiera, extranjero, media...
18658                                [quitar, sabor, boca]
18659                                      [isla, español]
18660                              [perdonir, inteligente]
Name: review_title, Length: 18659, dtype: object

In [42]:
combined_df['review_text'].apply(normalization)

0        [fan, serie, español, najwa, doler, serie, mal...
1        [malo, serie, poseer, respirar, diálogo, penos...
2        [serie, drama, médico, intentar, copiar, estil...
3        [obligar, querer, dinero, público, involucrado...
4        [serie, vergüenza, ajeno, serie, parecer, únic...
                               ...                        
18656    [film, serio, interpretación, estelar, fotogra...
18657    [esperar, guión, visto, novedoso, absoluto, pr...
18658    [cuerpo, sensación, finalizar, historia, inter...
18659    [añadir, comentario, guión, pésimo, actuación,...
18660    [película, cine, entusiasmo, prometer, sesión,...
Name: review_text, Length: 18659, dtype: object

In [45]:
combined_df.to_csv('../data/clean_data.csv', index=False)